# 초단위 단속 데이터 전처리

In [1]:
# module
import numpy as np
import pandas as pd
import tqdm
import requests

# 주소찾기
import time, random, re
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [2]:
# load data
data2020 = pd.read_csv('서울특별시_강서구_불법주정차단속자료_20201231.csv', encoding = 'cp949')
data2021 = pd.read_csv('서울특별시_강서구_불법주정차단속자료_20211231.csv', encoding = 'cp949')

In [3]:
# 데이터 합치기
data = pd.concat([data2020, data2021], ignore_index = True)

In [4]:
# 앞에 서울특별시 강서구 붙이기
data['단속장소'] = "서울특별시 강서구 " + data['단속장소'].str.replace('서울특별시 강서구', '').str.replace('서울 강서구', '')

In [5]:
# 단속 장소에서 부근 또는 주변 제거
data['단속장소'] = data['단속장소'].str.replace('부근', '').str.replace('주변', '').str.replace('오피스텔', '')

In [6]:
data2 = data.groupby(['단속장소']).count().reset_index().iloc[:, 0:2]

In [7]:
address = pd.DataFrame(index=range(0,0), columns = {'도로명 주소','지번 주소', '단속장소', 'count'})
address.columns = ['단속장소', 'count', '도로명주소', '지번주소']

def apt_juso(keyword, juso_cnt, cnt):
    ## 추출한 주소 저장할 엑셀 생성
    theme_list = ['No', '우편번호', '도로명 주소', '도로명 영문주소', '지번 주소']

    ## 주소 추출하기
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
    }

    ext_cnt = 1 # int(juso_cnt//10) + 1
    row = 2
    for page in range(1, ext_cnt + 1):
        url = f'https://www.juso.go.kr/support/AddressMainSearch.do?currentPage={page}&countPerPage=10&&searchType=\
        TOTAL&searchKeyword={keyword}&firstSort=none&ablYn=Y&synnYn=N'

        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        ol_lis = soup.select('#searchAddress > div.container.support_search_list > div.search_list > ol > li')

        for num, li in enumerate(ol_lis):
            if num == 1:
                break
    
            roed_juso = li.find('div', class_='subejct_1').find('span', class_='roadNameText').text
            road = re.sub('&nbsp;|\t|\r|\n', '', roed_juso).strip().replace('  ', ' ')
            # print(f'\n[{row-1}] 주소추출:\n{road}')

            road_eng = li.find('div', class_='addrEngInfo').find('span', class_='addrEng').text
            eng_roadname = re.sub('&nbsp;|\t|\r|\n', '', road_eng).strip().replace('  ', ' ')
            # print(eng_roadname)                

            jibeon_juso = li.find('div', class_='subejct_2').find('span', class_='roadNameText').text
            jibeon = re.sub('&nbsp;|\t|\r|\n', '', jibeon_juso).strip().replace('  ', ' ')
            # print(jibeon)

            zipcode = li.select_one('div.addrWrap > div.zipcode > div > strong').text
            # print(zipcode)

            btn = li.find('button', class_='btn_dtaddr_on')
            
        if ol_lis:
            address.loc[len(address)] = [keyword, cnt, road, jibeon]
        else:
            address.loc[len(address)] = [keyword, cnt, "no", "no"]
                
        time.sleep(random.uniform(0.5, 1))

In [10]:
for j, k in tqdm.tqdm(zip(data2['단속장소'][2711:], data2['단속일'][2711:])):
    apt_juso(j, 4, k)

23954it [7:34:21,  1.14s/it]


In [24]:
address['도로명주소'] = address['도로명주소'].str.replace(u'\xa0', u' ')

In [31]:
address.to_csv('불법주정차.csv', index = False, encoding = 'cp949')

In [30]:
address

,단속장소,count,도로명주소,지번주소
0,서울특별시 강서구 강서로 화곡동941-1,1,no,no
1,서울특별시 강서구 강서로5나길 화곡동949-7,1,no,no
2,서울특별시 강서구 194-15,1,서울특별시 강서구 화곡로 194-15 (화곡동),서울특별시 강서구 화곡동 98-238
3,서울특별시 강서구 5단지상가,1,no,no
4,서울특별시 강서구 가로공원로 173,17,서울특별시 강서구 가로공원로 173 (화곡동),서울특별시 강서구 화곡동 377-42
...,...,...,...,...
26660,서울특별시 강서구 화골로63길96,1,no,no
26661,서울특별시 강서구 화일초교,207,서울특별시 강서구 까치산로10길 70 (화곡동),서울특별시 강서구 화곡동 61-184 서울화일초등학교
26662,서울특별시 강서구 화일초등학교,397,서울특별시 강서구 까치산로10길 70 (화곡동),서울특별시 강서구 화곡동 61-184 서울화일초등학교
26663,서울특별시 강서구 화천종합연구센터,3049,서울특별시 강서구 마곡중앙8로5길 2 (마곡동),서울특별시 강서구 마곡동 792-8 화천종합연구센터
